## 1.1 Modelado del Sistema de Recomendación

In [ ]:
!pip install apyori

In [ ]:
####MODELADO DEL SISTEMA DE RECOMENDACIÓN
### REGLAS DE ASOCIACIÓN
## método a priori
#from mlxtend.frequent_patterns import apriori, association_rules
import numpy as np
import statistics
import pandas as pd
from apyori import apriori

## 1.2 Montar Google Drive

El presente notebook está pensado para ser ejecutado en Google Colaboratory y leyendo la información desde la cuenta de Google Drive del usuario.

Sin embargo si el usuario desea ejecutarlo en su equipo local, deberá modificar apropiadamente las variables definidas en la sección 1.3 (específicamente los paths de los directorios) y omitir la ejecución de la siguiente celda.

In [ ]:
# Punto de montaje remoto (Google Drive)
path_drive = '/content/drive/' 

## PATH donde se encuentran los Datasets
path_dataset = path_drive + 'My Drive/proyecto/3. Preparación de los datos' 
print('path_dataset:', path_dataset)

# Cargar Datos de
# https://colab.research.google.com/notebooks/io.ipynb
from google.colab import drive

# Montar Google Drive (ingresar credenciales de: g1.recomendaciones@gmail.com)
drive.mount(path_drive)

path_dataset: /content/drive/My Drive/proyecto/3. Preparación de los datos


ModuleNotFoundError: ignored

## 1.3 Lectura de las bases

> Bloc con sangría



In [ ]:
dataset = path_dataset + '/DataClean.parquet'
print(dataset)

update_df = pd.read_parquet(dataset, engine='auto')
# 11,3Millones de registros aproximadamente

In [ ]:
update_df.shape


## 1.4 Preparación de los datos

In [ ]:
print(update_df['event_type'].value_counts())

In [ ]:
update_df.groupby(by="event_type").sum()

In [ ]:
q0 = update_df['price'].quantile(0)
q1= update_df['price'].quantile(0.25)
q2= update_df['price'].quantile(0.50)
q3= update_df['price'].quantile(0.75)
q4= update_df['price'].quantile(1)

print(q0,q1,q2,q3,q4)

[0 - 100], [100 - 200], [200 - 500], [500 -  2574]

In [ ]:
update_df.head(2)

In [ ]:
#print( update_df[['user_id', 'brand','rank_precio', 'product_name', 'region', 'provincia', 'ciudad', 'sexo' ]].head(2))

transaction=  update_df[['user_id', 'brand','product_name', 'region', 'provincia', 'ciudad', 'sexo','price' ]] # agregar la coluna de precio

transaction = transaction[transaction['brand'] !="NA" ]

transaction = transaction[transaction['product_name'] !="ERROR" ]


In [ ]:
transaction.shape
#(9133762, 8)


update_df.head(4)

NameError: ignored

In [ ]:
tmp= transaction.groupby(['brand','product_name', 'ciudad', 'sexo'], as_index=False )["price"].sum()


In [ ]:
tmp= pd.DataFrame(tmp)

In [ ]:
tmp.head(5)
#t.head(5)

tmp.shape
#(315958, 5)


(315958, 5)

In [ ]:
tmp = tmp.drop(columns=['price'])

In [ ]:
tmp.shape

(315958, 4)

In [ ]:
tmp.head(3)

,brand,product_name,ciudad,sexo
0,a-case,soldadura,ABDÓN CALDERÓN (LA UNION) ...,F
1,a-case,soldadura,ALLURIQUÍN ...,F
2,a-case,soldadura,ALOAG ...,F


In [ ]:
res = pd.get_dummies(tmp )

In [ ]:
res.head()

,brand_a-case,brand_a-derma,brand_a-elita,brand_a-mega,brand_a-toys,brand_aardwolf,brand_abaybooks,brand_abc,brand_abhika,brand_abk,brand_abris,brand_absolutechampion,brand_abtoys,brand_abugarcia,brand_academie,brand_acase,brand_accesstyle,brand_accumaster,brand_acd,brand_acer,brand_aces,brand_achilles,brand_acm,brand_acme,brand_acnecontrolprofessional,brand_acorp,brand_acqua,brand_acron,brand_action,brand_actiontrack,brand_active,brand_activision,brand_acuvue,brand_acv,brand_adagio,brand_adam,brand_adamas,brand_adamex,brand_adata,brand_addlink,...,ciudad_TUMBACO,ciudad_TUNDAYME,ciudad_TUPIGACHI,ciudad_TURI,ciudad_TUTUPALI,ciudad_UNAMUNCHO,ciudad_URBINA (TAYA),ciudad_URCUQUI,ciudad_URDANETA (PAQUISHAPA),ciudad_UYUMBICHO,ciudad_VALDEZ (LIMONES),ciudad_VALENCIA,ciudad_VALLE,ciudad_VALLE HERMOSO,ciudad_VELASCO IBARRA,ciudad_VENTANAS,ciudad_VICENTINO,ciudad_VICHE,ciudad_VICTORIA DEL PORTETE (IRQUIS),ciudad_VILCABAMBA (VICTORIA),ciudad_VILLA LA UNIÓN (CAJABAMBA),ciudad_VINCES,ciudad_VIRGEN DE FÁTIMA,ciudad_YAGUACHI VIEJO (CONE),ciudad_YANGANA (ARSENIO CASTILLO),ciudad_YANTZAZA,ciudad_YARUQUÍ,ciudad_YUNGANZA (CAB. EN EL ROSARIO),ciudad_ZAMBIZA,ciudad_ZAMORA,ciudad_ZAPOTAL,ciudad_ZAPOTILLO,ciudad_ZARUMA,ciudad_ZHIDMAD,ciudad_ZUMBA,ciudad_ZUMBAHUA,ciudad_ZUMBI,ciudad_ZURMI,sexo_F,sexo_M
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
res.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 315958 entries, 0 to 315957
Columns: 4863 entries, brand_a-case to sexo_M
dtypes: uint8(4863)
memory usage: 1.4 GB


In [ ]:
#res.astype(np.int64)

In [ ]:
def encode_units(x):
    return int(x)

trans_set = res.applymap(encode_units)


In [ ]:
 #frq_items = apriori(res, min_support = 0.10,min_confidence = 0.5, use_colnames = True, min_lenght=5)
 frq_items = apriori(res, min_support = 0.10, use_colnames = True)
 

In [ ]:
#rules = association_rules(frq_items, metric="lift", min_threshold=1)
#rules.head()

rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules['antecedents'] = [str(element.split('{')[1].split('}')[0]).replace("'", '') for element in rules['antecedents'].astype("string").tolist()]
rules['consequents'] = [str(element.split('{')[1].split('}')[0]).replace("'", '') for element in rules['consequents'].astype("string").tolist()]

AttributeError: ignored

In [ ]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
66,ciudad_GUAYAQUIL ...,provincia_GUAYAS ...,0.315,0.343,0.315,1.0,2.915452,0.206955,inf
216,ciudad_GUAYAQUIL ...,provincia_GUAYAS ...,0.315,0.343,0.315,1.0,2.915452,0.206955,inf
219,ciudad_GUAYAQUIL ...,"user_id, provincia_GUAYAS ...",0.315,0.343,0.315,1.0,2.915452,0.206955,inf
319,ciudad_GUAYAQUIL ...,provincia_GUAYAS ...,0.100,0.343,0.100,1.0,2.915452,0.065700,inf
364,"product_name_teléfono inteligente, ciudad_GUAY...",provincia_GUAYAS ...,0.189,0.343,0.189,1.0,2.915452,0.124173,inf


In [ ]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [ ]:
from collections import Counter
userPurchases = Counter(update_df['user_id'].tolist())

In [ ]:
allBrand = Counter(update_df['brand'].tolist())
mean = statistics.mean([allBrand[nameProduct] for nameProduct in allBrand.keys() ])

In [ ]:
[nameProduct for nameProduct in allBrand.keys() if allBrand[nameProduct] > mean][:3]

['samsung', 'apple', 'xiaomi']

In [ ]:
def createBasket(df, brandName):
  basket = (df[df['brand'] == brandName]
          .groupby(['product_id', 'product_name'])['price']
          .sum().unstack().reset_index().fillna(0)
          .set_index('product_id'))
  return basket

def createUserBasket(df, userID):
  basket = (df[df['user_id'] == userID]
          .groupby(['brand', 'product_name'])['price']
          .sum().unstack().reset_index().fillna(0)
          .set_index('brand'))
  return basket

def hot_encode(x):
  if (x <= 0):
    return 0
  if (x >= 1):
    return 1

In [ ]:
basketSamsung = createBasket(update_df, 'apple')
basket_encoded = basketSamsung.applymap(hot_encode)

In [ ]:
basket_encoded.head(3)

product_name,ERROR,auricular,bicicleta,bolsa,bufanda,calcetín,columpio,computadora portátil,cultivador,escritorio,esquí,fuente de alimentación,libros electrónicos,luz,mesa,ratón,refrigeradores,reloj,slipones,soldadura,tableta,teclado,teléfono inteligente
product_id,,,,,,,,,,,,,,,,,,,,,,,
100001259,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100013611,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
100013738,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
basketSamsung.info()

<class 'pandas.core.frame.DataFrame'>
Index: 627 entries, 100001259 to 9200775
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ERROR                   627 non-null    float32
 1   auricular               627 non-null    float32
 2   bicicleta               627 non-null    float32
 3   bolsa                   627 non-null    float32
 4   bufanda                 627 non-null    float32
 5   calcetín                627 non-null    float32
 6   columpio                627 non-null    float32
 7   computadora portátil    627 non-null    float32
 8   cultivador              627 non-null    float32
 9   escritorio              627 non-null    float32
 10  esquí                   627 non-null    float32
 11  fuente de alimentación  627 non-null    float32
 12  libros electrónicos     627 non-null    float32
 13  luz                     627 non-null    float32
 14  mesa                    627 non-nul

In [ ]:
def createRules(df, brandName):
  basketSamsung = createBasket(df, brandName)
  basket_encoded = basketSamsung.applymap(hot_encode)
  basketSamsung = basket_encoded
  basketSamsung.info()
  frq_items = apriori(basketSamsung, min_support = 0.10, use_colnames = True)
  rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
  rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
  rules['antecedents'] = [str(element.split('{')[1].split('}')[0]).replace("'", '') for element in rules['antecedents'].astype("string").tolist()]
  rules['consequents'] = [str(element.split('{')[1].split('}')[0]).replace("'", '') for element in rules['consequents'].astype("string").tolist()]

  rules.to_csv(brandName+'.csv')
  return rules

In [ ]:
update_df.head(3)

In [ ]:
createRules(update_df, 'apple')
#createRules(update_df, 'huawei')
#createRules(update_df, 'lg')
#createRules(update_df, 'samsung')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,luz,teléfono inteligente,0.220096,0.296651,0.213716,0.971014,3.273259,0.148425,24.265550
4,slipones,tableta,0.135566,0.146730,0.106858,0.788235,5.371995,0.086966,4.029328
0,ERROR,soldadura,0.189793,0.204147,0.146730,0.773109,3.787027,0.107985,3.507650
5,tableta,slipones,0.146730,0.135566,0.106858,0.728261,5.371995,0.086966,3.181116
3,teléfono inteligente,luz,0.296651,0.220096,0.213716,0.720430,3.273259,0.148425,2.789658
1,soldadura,ERROR,0.204147,0.189793,0.146730,0.718750,3.787027,0.107985,2.880737


In [ ]:
userBasket = createUserBasket(update_df, '571290217')
basket_encoded = userBasket.applymap(hot_encode)
userBasket = basket_encoded

In [ ]:
print(userBasket)

Empty DataFrame
Columns: []
Index: []
